In [1]:
import csv 

csv_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/classification_dataset.csv'
with open(csv_path, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    dataset = []
    for row in reader:
        dataset.append(row)
del dataset[0] #remove header !

In [1]:
import tensorflow as tf 
model = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/training/20210621-14:00:08/trained_model_20210621-14:00:08.h5'
image = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/training/20210621-14:00:08.model.png'
tf.keras.utils.plot_model(model, to_file=image)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [2]:
dataset[0]

['1.2.276.0.7230010.3.2.212610',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/CLASSIFICATION/image_a_annoter/gained/1.2.276.0.7230010.3.2.212610_mip_ct.png',
 'Vertex',
 'Hips',
 'up',
 'up']

In [6]:
len(dataset)

2176

In [3]:
csv_path_ahl = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/AHL_NIFTI/AHL2011_NIFTI_V5.csv'
with open(csv_path_ahl, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    dataset_ahl = []
    for row in reader:
        dataset_ahl.append(row)
del dataset_ahl[0] #remove header !


csv_path_gained = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/GAINED_NIFTI/GAINED_PET0_NIFTI_V2.csv'
with open(csv_path_gained, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    dataset_gained = []
    for row in reader:
        dataset_gained.append(row)
del dataset_gained[0] #remove header !

In [5]:
dataset_reunite = dataset_ahl+dataset_gained 
print(dataset_reunite[0])

['CON,DA', '13011101331015', '1.2.124.113532.10.149.172.5.20131107.93455.7725981', 'pet0', 'AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_CT.nii', 'AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_PT.nii', 'AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_mask_3d.nii']


In [7]:

for row in dataset : 
    study_uid = row[0]
    for patient in dataset_reunite : 
        if study_uid == patient[2] : 
            row.append(patient[4]) #ajouter chemin CT nifti 

In [9]:
for row in dataset :
    if len(row) != 7 : 
        print(row)

ing/78ca2911-9e9f-4f78-b80a-848024b95f92/CLASSIFICATION/image_a_annoter/gained_pet4/2.16.56.465769650.2551326252.2444644315_mip_ct.png', 'Vertex', 'Hips', 'down', 'down']
['1.2.840.113711.3413733.332952.453985595.26.2116281012.10', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/CLASSIFICATION/image_a_annoter/gained_pet2/1.2.840.113711.3413733.332952.453985595.26.2116281012.10_mip_ct.png', 'Eye', 'Hips', 'up', 'up']
['1.2.250.1.74.20150602082500.4000021432846', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/CLASSIFICATION/image_a_annoter/gained_pet4/1.2.250.1.74.20150602082500.4000021432846_mip_ct.png', 'Mouth', 'Hips', 'up', 'up']
['1.2.250.1.74.20140513140000.1000041384511', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/CLASSIFICATION/image_a_annoter/gained_pet4/1.2.250.1.74.20140513140000.1000041384511_mip_ct.png', 'Vertex', 'Hips', 'down', 'down']
['2.16.56.465769650.2551326252.2361160945', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/C

In [ ]:
directory = ''
filename = 'classification_dataset.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_ID", "STUDY_UID", "NIFTI_CT", "RIGHT_ARM", "LEFT_ARM", "HEAD", "LEGS"])
    for serie in dataset: 
        #serie = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, nifti_CT, nifti_PET, nifti_MASK]
        csv_writer.writerow([   ])